In [1]:
import pickle
from PIL import Image
import numpy as np
import gradio as gr
from pathlib import Path
from transformers import pipeline
from tensorflow.keras.models import load_model
import tensorflow as tf
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import gradio as gr
from dotenv import load_dotenv
import openai
import os
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI


/Users/KTSnyder/ASU-VIRT-AI-PT-11-2023-U-LOLC/.conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the model's file path
file_path = Path("models/model_adam_scaled.h5")

# Load the model to a new object
adam_5 = tf.keras.models.load_model(file_path)

In [3]:
# Model summary
adam_5.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 394272)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │    25,233,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,234,435 (96.26 MB)

 Trainable params: 25,234,433 (96.26 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [4]:
# Load env variables    
load_dotenv()

# Add your OpenAI API key here
openai_api_key = os.getenv("OPENAI_API_KEY")

print (f"OpenAI API Key Loaded: {openai_api_key is not None}")

OpenAI API Key Loaded: True


In [5]:
# Create instance
llm = ChatOpenAI(openai_api_key=openai_api_key, model_name='gpt-3.5-turbo', temperature=0)

/Users/KTSnyder/ASU-VIRT-AI-PT-11-2023-U-LOLC/.conda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
# Define the ChatGPT function using the OpenAI API
def chatgpt(input_text):
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": input_text}
            ]
        )
        return response.choices[0].message['content'].strip()
    except Exception as e:
        print(f"Error in ChatGPT request: {e}")
        return "There was an error processing the chat request. Please try again."
# Placeholder functions (should be defined with actual implementation)
def preprocess_image(img):
    img = img.resize((224, 224))
    img_array = np.array(img) / 255.0
    img_array = img_array[np.newaxis, ...]
    return img_array

def interpret_prediction(prediction):
    return "Prediction interpretation placeholder"

In [7]:
# Load the model and tokenizer for translation
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
sorted_languages = sorted(tokenizer.lang_code_to_id.keys())
selected_code = tokenizer.lang_code_to_id["en_XX"]


# Set source language
tokenizer.src_lang = 'en_XX'

In [8]:
# Placeholder functions (should be defined elsewhere in your notebook)
def preprocess_image(img):
    # Preprocess the image before prediction
    pass

def interpret_prediction(prediction):
    # Interpret the model's prediction
    return "Prediction interpretation placeholder"

In [9]:
# Method to get system and human messages for ChatOpenAI
def get_messages(prediction_text):
    # Create a HumanMessage object
    human_message = HumanMessage(content=f'skin lesion that appears {prediction_text}')
    
    # Get the system message
    system_message =  SystemMessage(content='You are a medical professional chatting with a patient. You want to provide helpful information and give a preliminary assessment.')
    
    # Return the system message
    return [system_message, human_message]

In [10]:
# Language information MBart
language_info = [
    "English (en_XX)", "Arabic (ar_AR)", "Czech (cs_CZ)", "German (de_DE)",
    "Spanish (es_XX)", "Estonian (et_EE)", "Finnish (fi_FI)", "French (fr_XX)",
    "Gujarati (gu_IN)", "Hindi (hi_IN)", "Italian (it_IT)", "Japanese (ja_XX)",
    "Kazakh (kk_KZ)", "Korean (ko_KR)", "Lithuanian (lt_LT)", "Latvian (lv_LV)",
    "Burmese (my_MM)", "Nepali (ne_NP)", "Dutch (nl_XX)", "Romanian (ro_RO)",
    "Russian (ru_RU)", "Sinhala (si_LK)", "Turkish (tr_TR)", "Vietnamese (vi_VN)",
    "Chinese (zh_CN)", "Afrikaans (af_ZA)", "Azerbaijani (az_AZ)", "Bengali (bn_IN)",
    "Persian (fa_IR)", "Hebrew (he_IL)", "Croatian (hr_HR)", "Indonesian (id_ID)",
    "Georgian (ka_GE)", "Khmer (km_KH)", "Macedonian (mk_MK)", "Malayalam (ml_IN)",
    "Mongolian (mn_MN)", "Marathi (mr_IN)", "Polish (pl_PL)", "Pashto (ps_AF)",
    "Portuguese (pt_XX)", "Swedish (sv_SE)", "Swahili (sw_KE)", "Tamil (ta_IN)",
    "Telugu (te_IN)", "Thai (th_TH)", "Tagalog (tl_XX)", "Ukrainian (uk_UA)",
    "Urdu (ur_PK)", "Xhosa (xh_ZA)", "Galician (gl_ES)", "Slovene (sl_SI)"
]

# Convert the information into a dictionary
language_dict = {}
for info in language_info:
    name, code = info.split(' (')
    code = code[:-1]  
    language_dict[name] = code

# Get the language names for choices in the dropdown
languages = list(language_dict.keys())
first_language = languages[0]
sorted_languages = sorted(languages[1:])
sorted_languages.insert(0, first_language)

default_language = 'English'

# Prediction responses
malignant_text = "Malignant. Please consult a doctor for further evaluation."
benign_text = "Benign. Please consult a doctor for further evaluation."

def submit(language, img):
    print(f'Language: {language}')
    if language is None or len(language) == 0:
        language = default_language
    if img is None:
        return 'No image uploaded. Please try again.'
    return predict_image(language, img)

def predict_image(language, img):
    try:       
        try: 
            # Process the image
            img = img.resize((224, 224))
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
        except Exception as e:
            print(f'Error: {e}')
            return 'There was an error processing the image. Please try again.'
        
        # Get prediction from model
        prediction = adam_5.predict(img_array)
        text_prediction = 'Malignant' if prediction[0][0] > 0.5 else 'Benign'
        
        try:
            # Get the system and human messages
            messages = get_messages(text_prediction)
            
            # Get the response from ChatOpenAI
            result = llm(messages)
            
            # Get the text prediction
            text_prediction = result.content
            
        except Exception as e:
            print(f'Error: {e}')
            print(f'Prediction: {text_prediction}')
            text_prediction = malignant_text if text_prediction == 'Malignant' else benign_text 

        # Get selected language code
        selected_code = language_dict[language]
        
        # Check if the target and source languages are the same
        if selected_code == 'en_XX':
            return text_prediction
        
        try:
            # Encode, generate tokens, decode the prediction
            encoded_text = tokenizer(text_prediction, return_tensors="pt")
            generated_tokens = model.generate(
                **encoded_text,
                forced_bos_token_id=tokenizer.lang_code_to_id[selected_code]
            )
            result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            
            # Return the result
            return result[0]
        except Exception as e:
            print(f'Error: {e}')
            return f'''There was an error processing the translation. 
            In English:
            {text_prediction}
            '''
        
    except Exception as e:
        print(f'Error: {e}')
        return 'There was an error processing the request. Please try again.'
    
# Define the chat function to handle user queries
def chat(chat_prompt):
    print(f"Chat prompt received: {chat_prompt}")
    if chat_prompt:
        chat_response = chatgpt(chat_prompt)
        print(f"ChatGPT response: {chat_response}")
        return chat_response
    return "Please enter a prompt to chat."
    
    
# Redefine the Gradio interface to include the chat functionality
with gr.Blocks(theme=gr.themes.Default(primary_hue="green")) as demo:
    intro = gr.Markdown(
    """
    # Welcome to Skin Lesion Image Classifier!
    Select prediction language and upload image to start.
    """)
    language = gr.Dropdown(
        label='Change Language - Default English',
        choices=sorted_languages
    )
    img = gr.Image(image_mode='RGB', type='pil')
    submit_btn = gr.Button('Submit', variant='primary')
    output = gr.Textbox(label='Results')
    chat_prompt = gr.Textbox(label='Do you have any questions about the results, skin cancer, where to find a dermotologist, or something else?', placeholder='Enter your question here...')
    chat_btn = gr.Button('Submit Question', variant='secondary')
    chat_output = gr.Textbox(label='Question Response')
    submit_btn.click(fn=submit, inputs=[language, img], outputs=output)
    chat_btn.click(fn=chat, inputs=[chat_prompt], outputs=chat_output)
    clear_btn = gr.ClearButton(components=[language, img, output, chat_prompt, chat_output], variant='stop')


demo.launch()

Running on local URL:  http://127.0.0.1:7874

To create a public link, set `share=True` in `launch()`.


Language: None
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step


/Users/KTSnyder/ASU-VIRT-AI-PT-11-2023-U-LOLC/.conda/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Chat prompt received: Where is a dermotologist?
Error in ChatGPT request: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742

ChatGPT response: There was an error processing the chat request. Please try again.
